In [4]:
################################################################################
# MASTERARBEIT - SKRIPT 08:
# EXPERIMENT 2.2 (Forschungsfrage 2) - INKONSISTENZBEHEBUNG (LLM)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Abschnitt 3.3.2):
#
# 1. (Laden): Lädt den 'schmutzigen' Rohdatensatz (rfd_main.csv).
#
# 2. (Methode): Wendet die zweite Methode zur Inkonsistenzbehebung an:
#    Ein großes Sprachmodell (LLM), namentlich Claude 4.5 Sonnet.
#
# 3. (Ziel): Das Ziel ist die REPARATUR (Behebung) von Inkonsistenzen 
#    (Forschungsfrage 2), nicht nur deren Detektion (Forschungsfrage 1).
#
# 4. (Methodischer Ansatz - SEMANTISCHE REPARATUR):
#    Wir weisen das LLM an, den 'schmutzigen' Rohdatensatz (1326 Zeilen) 
#    Zeile für Zeile zu analysieren und ein "gereinigtes" (repariertes)
#    Gegenstück für JEDE Zeile zu generieren.
#
# 5. (Aufgaben der Reparatur): Der 'SYSTEM_PROMPT' wird das LLM anweisen, 
#    alle in der EDA (Skript 01) identifizierten Fehlertypen zu beheben:
#    a) Syntaktische Fehler: (z.B. 'price: "$19.99"' -> '19.99', 
#       'saving: "50%"' -> '0.5', 'price: "Free"' -> '0').
#    b) Logische Fehler: (z.B. 'replies: -1' -> '0' oder '1', 
#       ohne die Zeile zu entfernen).
#
# 6. (Speichern): Speichert das vom LLM komplett reparierte Dataset.
#    Dieses Ergebnis (z.B. 'rfd_repaired_llm.csv') dient als 
#    Grundlage für das Experiment der Forschungsfrage 3.
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import os
import sys 
import warnings
import json 
import time # Wird für die API-Schleife (Ratenlimits) benötigt

# Importieren der Anthropic-Bibliothek für die Claude-API
print("Lade notwendige Bibliotheken (Pandas, Numpy, OS, sys, json, time)...")
try:
    from anthropic import Anthropic, RateLimitError, APIError
    print("Anthropic (Claude) Bibliothek erfolgreich geladen.")
except ImportError as e:
    print(f"FEHLER: Kritische Bibliothek (Anthropic) konnte nicht geladen werden.")
    print(f"Stellen Sie sicher, dass 'pip install anthropic' ausgeführt wurde.")
    sys.exit("Skript gestoppt, da Abhängigkeiten fehlen.")

# Ignoriere technische Warnungen (für eine saubere Protokollausgabe)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

print("Alle Bibliotheken sind bereit.")
print("=" * 70)
#
#######################################################################################

# --- GLOBALE KONSTANTEN & PFADE ---
# (Inhalt unverändert; nur der Übersicht halber zentral gebündelt)
dateipfad = 'rfd_main.csv'
DATEI_AUSGABE = 'ergebnisse/2.2_rfd_repaired_llm_2_2.csv'
MODELL_NAME = "claude-sonnet-4-5-20250929"
#
#######################################################################################

# --- SCHRITT 2: DATEN LADEN & API-CLIENT INITIALISIEREN ---
print("\n--- Schritt 2: Daten laden & API-Client initialisieren ---")

# 2.1: Laden der Daten
try:
    df_schmutzig = pd.read_csv(dateipfad)
    print(f"Datensatz geladen: {dateipfad} (Dimensionen: {df_schmutzig.shape})")
except FileNotFoundError:
    print(f"FEHLER: Datei '{dateipfad}' nicht gefunden. Bitte Dateipfad prüfen.")
    sys.exit("Skript gestoppt.")

# 2.2: API-Client initialisieren
try:
    client = Anthropic() 
    # Modellname für die Reparatur (Reparatur ist rechenintensiv, daher Sonnet)
    print(f"Anthropic (Claude) API-Client und Modell '{MODELL_NAME}' initialisiert.")
except Exception as e:
    print(f"FEHLER: Der Anthropic API-Client konnte nicht initialisiert werden.")
    sys.exit("Skript gestoppt.")
print("-" * 40)

# 2.3: Datenvorbereitung (Kontext- & Zielspalten)
# Für die Reparatur (Inkonsistenzbehebung) benötigt das LLM alle Spalten 
# im Kontext, um semantische Entscheidungen treffen zu können.
alle_spalten = list(df_schmutzig.columns)

# Semantische Bereinigung: Konvertiere NaN-Werte in leere Strings für sauberen JSON-Input
df_llm_input = df_schmutzig.copy()
df_llm_input = df_llm_input.fillna('')
print("Datenvorbereitung abgeschlossen (NaN zu Leerstring konvertiert).")

print("=" * 70)
#
#######################################################################################

# --- SCHRITT 3: PROMPTS DEFINIEREN & HILFSFUNKTIONEN ---
print("\n--- Schritt 3: Prompt-Definition & Hilfsfunktionen ---")

# 3.1: Definition des SYSTEM_PROMPT (REPARATUR-LOGIK)
# DIES IST DER KERN DES EXPERIMENTS: Der Prompt instruiert das LLM zur Reparatur 
# syntaktischer und logischer Inkonsistenzen basierend auf der EDA und Methodik.
SYSTEM_PROMPT = """
Du bist ein führender Experte für Datenqualität und semantische Konsistenzprüfung 
mit Schwerpunkt auf **Geschäftslogik und Finanzdaten**. 
Deine Hauptaufgabe ist es, die dir vorgelegte einzelne tabellarische Zeile auf Basis 
deines tiefen **Domänenwissens** zu analysieren und die Datenqualität (DQ) im 
Sinne der **Zweckeignung** zu maximieren.

ZIEL: REPARATUR.
Analysiere die gesamte Zeile als EINEN semantischen Kontext. Gib das fehlerfreie 
und bereinigte Gegenstück der Zeile im JSON-Format zurück.

METHODIK:
1. **Beurteile den Kontext:** Identifiziere kritische Zusammenhänge zwischen Spalten 
   (z.B. die finanzielle Kette zwischen 'title', 'price' und 'saving').
2. **Logische Konsistenz:** Korrigiere Werte, die gegen Zählprinzipien (Negativität) 
   oder **temporale/kausale Logik** (z.B. Datumswerte) verstoßen.
3. **Semantische Korrektur (Finanzdaten):** Korrigiere numerische Werte und die 
   zugehörigen Formate so, dass die **interne mathematische Konsistenz** der Zeile 
   wiederhergestellt wird (z.B. Preis, Rabatt und Endpreis müssen logisch zueinander passen).
4. **Strukturelle Säuberung:** Entferne alle syntaktischen Störzeichen und stelle die 
   einheitliche, numerische Darstellung (Float) her.

AUSGABEFORMAT (ZWINGEND):
Gib deine Antwort NUR als EINZIGES, bereinigtes JSON-Objekt zurück, das 
ALLE ursprünglichen Spalten (Keys) enthält. Keine Erklärungen oder Texte.
"""

print("SYSTEM_PROMPT (Reparatur-Logik) definiert.")

# 3.2: Definition der USER_PROMPT Funktion
# Diese Funktion verpackt die zu reparierende Zeile in die Prompt-Struktur.
def erstelle_user_prompt_reparatur(daten_zeile):
    """
    Konvertiert eine Pandas Series (Zeile) in einen formatierten 
    JSON-String für den LLM-Prompt.
    """
    try:
        # Konvertiere die Pandas-Zeile in ein Dictionary (enthält alle Spalten)
        zeilen_dict = daten_zeile.to_dict()
        
        # Konvertiere das Dictionary in einen sauberen JSON-String
        zeilen_json = json.dumps(zeilen_dict, indent=2, ensure_ascii=False)
        
        # Erstelle den finalen Prompt
        user_prompt = f"""
Hier ist eine Zeile mit möglicherweise inkonsistenten Daten. 
Gib die bereinigte Version im JSON-Format zurück.

DATENZEILE ZUR REPARATUR:
{zeilen_json}
"""
        return user_prompt

    except Exception as e:
        print(f"FEHLER beim Erstellen des User-Prompts: {e}")
        return None

print("Funktion 'erstelle_user_prompt_reparatur' definiert.")

# 3.3: HILFSFUNKTION: API-AUFRUF MIT BACKOFF (muss VOR Schritt 4 stehen)
def rufe_claude_api_mit_backoff(system_prompt, user_prompt, model, max_retries=5):
    """
    Führt den Anthropic API-Aufruf mit Exponential Backoff bei Ratenlimit-Fehlern durch.
    """
    # client wird als globale Variable aus SCHRITT 2 verwendet
    
    for attempt in range(max_retries):
        try:
            response = client.messages.create(
                model=model,
                max_tokens=2048, 
                system=system_prompt,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            
            # WICHTIG: Claude liefert den reinen JSON-Text in response.content[0].text
            raw_json_text = response.content[0].text.strip()
            
            # Überprüfung und Extraktion des JSON-Objekts (robust)
            if raw_json_text.startswith('{') and raw_json_text.endswith('}'):
                return json.loads(raw_json_text)
            else:
                # Notfall-Logik (sucht nach dem ersten '{' und letzten '}')
                start = raw_json_text.find('{')
                end = raw_json_text.rfind('}')
                
                if start != -1 and end != -1 and end > start:
                    extracted_json = raw_json_text[start:end+1]
                    return json.loads(extracted_json)
                else:
                    raise json.JSONDecodeError("JSON nicht extrahierbar.", raw_json_text, 0)
            
        except RateLimitError:
            delay = 2 ** attempt # Exponential Backoff (1s, 2s, 4s, 8s, 16s, ...)
            print(f"\nAPI-Ratenlimit erreicht. Warte {delay} Sekunden vor Wiederholung (Versuch {attempt + 1}/{max_retries}).")
            time.sleep(delay)
            continue
            
        except APIError as e:
            # Behandlung anderer API-Fehler (z.B. Authentifizierung, die wir bereits behoben haben)
            print(f"\nAPI-Fehler bei Versuch {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(5) 
                continue
            else:
                return None
        except Exception as e:
            # Fehler (JSONDecodeError oder andere)
            return None
            
    return None

print("=" * 70)
#
#######################################################################################

# --- SCHRITT 4: TESTLAUF DER KRITISCHEN ZEILE (Index 7) ---
print("\n--- Schritt 4: Testlauf der kritischen Zeile (Index 7) ---")

# Methodisches Ziel: Überprüfen, ob das LLM die komplexe semantische/finanzielle 
# Logik (278+HST, 20% Coupon und $119.60 Saving) in Index 7 korrekt auflöst.
# Dies ist der ultimative Test des aktuellen SYSTEM_PROMPT.

# 1. INITIALISIERUNG: Listen müssen existieren (zur Behebung des NameError)
reparierte_datensaetze = []
fehler_indizes_reparatur = [] 

KRITISCHE_INDEX = 7

try:
    print(f"Starte API-Aufruf für Index {KRITISCHE_INDEX}...")
    
    # a) Kritisches Zeile extrahieren
    test_row = df_llm_input.iloc[KRITISCHE_INDEX]
    
    # b) Prompt erstellen
    test_user_prompt = erstelle_user_prompt_reparatur(test_row)
    
    # c) API-Aufruf (Mit der robusten Funktion rufe_claude_api_mit_backoff)
    # Hier verwenden wir die robuste Funktion, falls Ratenlimits auftreten.
    ergebnis_dict = rufe_claude_api_mit_backoff(SYSTEM_PROMPT, test_user_prompt, MODELL_NAME)
    
    if ergebnis_dict:
        # d) Sonucu listeye kaydet
        ergebnis_dict['original_index'] = KRITISCHE_INDEX
        reparierte_datensaetze.append(ergebnis_dict)
        print(f"STATUS: Index {KRITISCHE_INDEX} erfolgreich repariert und gespeichert.")
    else:
        # e) Fallback (Hata durumunda)
        fehler_indizes_reparatur.append(KRITISCHE_INDEX)
        print(f"FEHLER: Index {KRITISCHE_INDEX} konnte nicht repariert werden (API/JSON Error).")
        
except Exception as e:
    print(f"KRITISCHER FEHLER IM TESTLAUF: {e}")
    sys.exit("Skript gestoppt.")

print("=" * 70)
#
#######################################################################################

# --- SCHRITT 5: Reparierte Daten in DataFrame umwandeln und speichern ---
print("\n--- Schritt 5: Reparierte Daten in DataFrame umwandeln und speichern ---")

if len(reparierte_datensaetze) == 0:
    print("FEHLER: Keine reparierten Ergebnisse zum Speichern vorhanden. Das Skript muss in Schritt 4 fehlgeschlagen sein.")
    sys.exit("Skript gestoppt.")

# 1. Konvertiere die Liste der reparierten JSON-Antworten in einen Pandas DataFrame
df_repaired_llm = pd.DataFrame(reparierte_datensaetze)

# Überprüfung, ob die Spalte 'original_index' existiert (wichtig für den Fallback)
if 'original_index' in df_repaired_llm.columns:
    # 2. Setze den 'original_index' als Hauptindex 
    df_repaired_llm = df_repaired_llm.set_index('original_index').sort_index()
    print(f"Reparierter Datensatz ({len(df_repaired_llm)} Zeilen) erfolgreich erstellt.")
else:
    # Dies sollte nur passieren, wenn alle Zeilen in Schritt 4 fehlschlagen, was sehr unwahrscheinlich ist.
    print("WARNUNG: 'original_index' Spalte nicht gefunden. Index wird neu gesetzt.")
    df_repaired_llm.index.name = 'original_index'

# 3. Definiere die Ausgabe-Datei und erstelle den Ordner, falls nötig
os.makedirs(os.path.dirname(DATEI_AUSGABE), exist_ok=True)

# 4. Speichern des reparierten Datensatzes
try:
    df_repaired_llm.to_csv(DATEI_AUSGABE, index=True)
    
    print("-" * 40)
    print(f"AD HOC ANALYSE: Anzahl Zeilen im bereinigten Datensatz: {len(df_repaired_llm)}")
    print(f"ERGEBNIS (LLM): Reparierte Daten erfolgreich gespeichert in: '{DATEI_AUSGABE}'")
    print("-" * 40)

except Exception as e:
    print(f"FEHLER beim Speichern der reparierten Daten: {e}")
    sys.exit("Skript gestoppt.")

#
#######################################################################################

# --- SCHRITT 6: Zusammenfassung (Protokoll) ---
print("\n--- Schritt 6: Zusammenfassung EXPERIMENT 2.2 (LLM Reparatur) ---")
# Wir nehmen an, dass 'df_schmutzig' global geladen wurde (Schritt 2)
if 'df_schmutzig' in globals():
    initial_shape = df_schmutzig.shape
else:
    # Fallback, falls df_schmutzig nicht definiert ist (sehr unwahrscheinlich)
    initial_shape = "Unbekannt"

# Angabe der fehlgeschlagenen Zeilen (muss in Schritt 4 initialisiert werden)
if 'fehler_indizes_reparatur' not in globals():
    # Wenn die Variable nicht existiert (da Schritt 4 die Listenstruktur geändert hat), 
    # berechnen wir die übersprungenen Zeilen als Differenz zur Gesamtzahl.
    # Dabei wird davon ausgegangen, dass 'df_schmutzig' definiert ist.
    if isinstance(initial_shape, tuple):
        uebersprungen = initial_shape[0] - len(reparierte_datensaetze)
    else:
        uebersprungen = "Unbekannt (df_schmutzig nicht gefunden)"
elif isinstance(fehler_indizes_reparatur, list):
    uebersprungen = len(fehler_indizes_reparatur) # Sollte 0 sein im aktuellen Code
else:
    uebersprungen = "Fehlerhafte Protokollierung"

print(f"Methode:             Moderne KI: LLM (Claude 4.5 Sonnet)")
print(f"Zieldaten:           {dateipfad} (Initial Shape: {initial_shape})")
print(f"Aufgabe:             Semantische und Strukturelle Reparatur (FF2)")
print(f"Finaler Datensatz:   {df_repaired_llm.shape}")
print(f"Übersprungene Zeilen: {uebersprungen}")
print("-" * 40)
print("ERFOLG: LLM-Reparatur abgeschlossen.")
print("Diese Datei dient als Grundlage für die FF3-Experimente.")
print("=" * 70)
#
#######################################################################################


Lade notwendige Bibliotheken (Pandas, Numpy, OS, sys, json, time)...
Anthropic (Claude) Bibliothek erfolgreich geladen.
Alle Bibliotheken sind bereit.

--- Schritt 2: Daten laden & API-Client initialisieren ---
Datensatz geladen: rfd_main.csv (Dimensionen: (1326, 15))
Anthropic (Claude) API-Client und Modell 'claude-sonnet-4-5-20250929' initialisiert.
----------------------------------------
Datenvorbereitung abgeschlossen (NaN zu Leerstring konvertiert).

--- Schritt 3: Prompt-Definition & Hilfsfunktionen ---
SYSTEM_PROMPT (Reparatur-Logik) definiert.
Funktion 'erstelle_user_prompt_reparatur' definiert.

--- Schritt 4: Testlauf der kritischen Zeile (Index 7) ---
Starte API-Aufruf für Index 7...
STATUS: Index 7 erfolgreich repariert und gespeichert.

--- Schritt 5: Reparierte Daten in DataFrame umwandeln und speichern ---
Reparierter Datensatz (1 Zeilen) erfolgreich erstellt.
----------------------------------------
AD HOC ANALYSE: Anzahl Zeilen im bereinigten Datensatz: 1
ERGEBNIS (LL

In [5]:
# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import os
import sys 

print("Lade notwendige Bibliotheken (Pandas, Numpy, OS, sys)...")
print("Alle Bibliotheken sind bereit.")
print("=" * 70)


dateipfad = 'ergebnisse/2.2_rfd_repaired_llm_2_2.csv' 
df_schmutzig = pd.read_csv(dateipfad)

print(f"Datensatz geladen: {dateipfad}")
print(f"Dimensionen (Zeilen, Spalten): {df_schmutzig.shape}")
print("=" * 70)

Lade notwendige Bibliotheken (Pandas, Numpy, OS, sys)...
Alle Bibliotheken sind bereit.
Datensatz geladen: ergebnisse/2.2_rfd_repaired_llm_2_2.csv
Dimensionen (Zeilen, Spalten): (1, 16)


In [6]:
# SCHRITT 2: Datentypen und Struktur prüfen
print("\n--- 2. Datentypen und Struktur (.info()) ---")
# Diese .info()-Ausgabe ist entscheidend. Sie bestätigt wissenschaftlich, 
# welche Spalten 'object' (Text) und welche 'int64'/'float64' (numerisch) sind.
# Dies beweist, dass traditionelle-Methoden (IQR) und Klasik ML-Methoden (LOF, Isolation Forest) auf 'price' oder 'saving'
# ohne eine Vorverarbeitung (wie in Forschungsfrage 2) nicht angewendet werden können.
df_schmutzig.info()
print("=" * 70)


--- 2. Datentypen und Struktur (.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   original_index   1 non-null      int64  
 1   Unnamed: 0       1 non-null      int64  
 2   author           1 non-null      object 
 3   creation_date    1 non-null      object 
 4   expiry           0 non-null      float64
 5   last_reply       1 non-null      object 
 6   parent_category  0 non-null      float64
 7   price            1 non-null      float64
 8   replies          1 non-null      int64  
 9   saving           1 non-null      float64
 10  source           1 non-null      object 
 11  thread_category  1 non-null      object 
 12  title            1 non-null      object 
 13  url              1 non-null      object 
 14  views            1 non-null      int64  
 15  votes            1 non-null      int64  
dtypes: float64(4), int64

In [7]:
# SCHRITT 3: Erste 10 Zeilen ansehen (.head(10)) 
print("\n--- 3. Erste 10 Zeilen (.head(10)) ---") 
# Dies zeigt visuell das Problem in den 'object'-Spalten.
df_schmutzig.head(30) 


--- 3. Erste 10 Zeilen (.head(10)) ---


,original_index,Unnamed: 0,author,creation_date,expiry,last_reply,parent_category,price,replies,saving,source,thread_category,title,url,views,votes
0,7,7,SoulTracer,2020-06-30T17:28:00,NaN,2020-07-17T09:14:00,NaN,278.4,157,119.6,Home Depot,Home & Garden,"40v 20"" Ryobi Push Mower $278+HST (w/ 20% coupon)",https://the-home-depot-ca.pxf.io/c/341376/5836...,20936,27
